# Implementing the SVM Algorithm

In [1]:
import pandas as pd
import numpy as np

df= pd.read_csv('../day5_titanic/train.csv')
df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [2]:
y=df['Survived']
X=df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked']]

In [3]:
X_numeric = X[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']]
X_categorical = X[['Sex','Embarked']]
X_neither = X['Cabin']
X_onehot = pd.get_dummies(X_categorical,['Sex','Embarked']).astype(int)
X_cabin_nonna = X['Cabin'].apply(lambda x: 0 if pd.isna(x) else 1)
X_encoded0 = pd.merge(X_numeric, X_cabin_nonna,  left_index=True, right_index=True)
X_encoded1 = pd.merge(X_encoded0, X_onehot,  left_index=True, right_index=True)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded1, y, test_size=0.2, random_state=42, stratify=y
)
from sklearn.impute import SimpleImputer

# Imputer for numeric columns (e.g., Age)
imputer = SimpleImputer(strategy='median')  # or 'mean' if you prefer

# Fit on train, transform train
X_train['Age'] = imputer.fit_transform(X_train[['Age']])

# Transform test using same statistics
X_test['Age'] = imputer.transform(X_test[['Age']])

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



### Application from sklearn library

In [6]:
# ----------------------------
# Import required libraries
# ----------------------------
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
# ----------------------------
# Create and train SVM model
# ----------------------------
# You can choose kernel='linear', 'rbf', 'poly', etc.
svm_model = SVC(kernel='rbf', C=0.1, gamma='scale', random_state=6)

# Train the model
svm_model.fit(X_train_scaled, y_train)
# ----------------------------
# Predictions
# ----------------------------
y_train_pred = svm_model.predict(X_train_scaled)
y_test_pred = svm_model.predict(X_test_scaled)

# ----------------------------
# Evaluate accuracy
# ----------------------------
train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)

print(f"Train Accuracy: {train_acc:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")


Train Accuracy: 0.8132
Test Accuracy: 0.8101


### Implementation from scratch

## Start here: X_train, X_test, y_train, y_test
### Hyperparameters: kernel, C, gamma

AFAIK the "math is analytical" - maximum margin classifier where margin is the distance of the separating hyperplane to its closest points - and only end results are in code

In [7]:
learning_rate = 0.01
kernel = 'linear'
C = 0.1

In [22]:

class SVM:
    def __init__(self,learning_rate=0.015, kernel='linear', C=0.1,n_iters=10000):
        self.C = C
        self.lr = learning_rate
        self.kernel = kernel
        self.n_iters=n_iters
        self.w = None
        self.b = None

    def fit(self, X, y):
        # gradient descent on the derivative of hinge loss 
        # hinge loss: 
        # first convert y 0,1 to y -1 / 1
        y_ = np.where(y <= 0, -1,1)
        n_samples, n_features = X.shape
        self.w = np.random.randn(n_features)
        self.b = np.random.randn(1)
        for _ in range(self.n_iters):
            # Compute the margin for all samples
            margins = y_ * (X.dot(self.w) + self.b)
            # Identify which samples violate the margin
            misclassified = margins < 1

            # Gradient for w and b
            dw = 2 * self.C * self.w - np.dot(X[misclassified].T, y_[misclassified]) / n_samples
            db = -np.sum(y_[misclassified]) / n_samples

            # Parameter update
            self.w -= self.lr * dw
            self.b -= self.lr * db



    def predict(self, X):
        # X is a numpy array n_examples * n_features
        # self.W is n_features * 1
        return np.sign (np.dot(X,self.w)+ self.b)

In [23]:
# Train the model
a = SVM(n_iters=20000)

a.fit(X_train_scaled, y_train)
# ----------------------------
# Predictions
# ----------------------------
# Convert back to {0, 1} for sklearn metrics
y_train_pred = np.where(a.predict(X_train_scaled) == -1, 0, 1)
y_test_pred = np.where(a.predict(X_test_scaled) == -1, 0, 1)
# ----------------------------
# Evaluate accuracy
# ----------------------------
train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)

print(f"Train Accuracy: {train_acc:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")


Train Accuracy: 0.7893
Test Accuracy: 0.7765


In [20]:
a.b

array([-0.29042782])

In [21]:
a.w

array([-0.00251015, -0.00369913, -0.00231407, -0.0006387 ,  0.00101648,
        0.00250444,  0.47768288, -0.47768288, -0.00080539,  0.00069612,
       -0.00273686])